In [16]:
from pylabnet.utils.logging.logger import LogClient

from pylabnet.hardware.zi_hdawg.zi_hdawg import Driver, Sequence, AWGModule

from pylabnet.utils.zi_hdawg_pulseblock_handler.zi_hdawg_pb_handler import DIOPulseBlockHandler

import pylabnet.utils.pulseblock.pulse as po
import pylabnet.utils.pulseblock.pulse_block as pb
from pylabnet.utils.pulseblock.pb_iplot import iplot
from pylabnet.utils.pulseblock.pb_sample import pb_sample

# Digital pulse playback using `pulseblock` and the DIO output of a ZI HDWAG

In [17]:
dev_id = 'dev8040'

# Instantiate logger.
logger = LogClient(
    host='192.168.1.2',
    port=3683,
    module_tag=f'ZI HDAWG {dev_id}'
)

# Instanciate HDAWG driver.
hd = Driver(dev_id, logger=logger)

Let's start with defining the digital part of a typical rabi-sequence. We use the `rabi_element` function as defined in the `pulseblock` demo notebook:

In [18]:
def rabi_element(tau=0, aom_offset=0):
    rabi_element = pb.PulseBlock(
        p_obj_list=[
            po.PTrue(ch='aom', dur=1.1e-6),
            po.PTrue(ch='ctr', t0=0.5e-6, dur=0.5e-6)
        ]
    )
    temp_t = rabi_element.dur

    rabi_element.insert(
        p_obj=po.PTrue(ch='mw_gate', dur=tau, t0=temp_t+0.7e-6)
    )
    temp_t = rabi_element.dur

    rabi_element.insert(
        p_obj=po.PTrue(ch='aom', t0=temp_t+aom_offset, dur=2e-6)
    )
    rabi_element.insert(
        p_obj=po.PTrue(ch='ctr', t0=temp_t, dur=0.5e-6)
    )

    rabi_element.dflt_dict = dict(
        aom=po.DFalse(),
        ctr=po.DFalse(),
        mw_gate=po.DFalse()
    )

    return rabi_element

In [19]:
# Let's choose a microwave duration of 1us.
tau = 1e-6

rabi_pulse = rabi_element(tau)

In [20]:
# Let's plot it.
iplot(rabi_pulse)

Now, let's turn this `pulseblock` instance into an intruction set for the DIO outputs of the HDAWG. We want to get a series on `setDIO()` and `wait()` commands, which reproduce the pulse sequence.

In [21]:
# Specify the DIO outputs which drive the MW, counter and aom.
assignment_dict = {
    'mw_gate':   15,
    'ctr':      17,
    'aom':      31,
}

In [22]:
# Instanciate pulseblock handler.
pb_handler = DIOPulseBlockHandler(
    pb = rabi_pulse,
    assignment_dict=assignment_dict,
    hd=hd
)

In [23]:
# Generate .seqc instruction set which represents pulse sequence.
dig_pulse_sequence = pb_handler.get_dio_sequence()

In [24]:
# Let's have a look.
print(dig_pulse_sequence)

setDIO(2147483648);wait(144);setDIO(2147614720);wait(147);setDIO(2147483648);wait(27);setDIO(0);wait(205);setDIO(32768);wait(296);setDIO(2147647488);wait(0);setDIO(2147614720);wait(146);setDIO(2147483648);wait(446);


This `.seqc` snippet can now be used in any sequence to reproduce the pulse sequence above. Let's try to write a simple sequence which outputs the digital pulse sequence as well as a trigger signal, and let's look at the results on the scope. 

To do so, connect marker output 1 of the HDWAG to the scope and use it as scope trigger, end then connect DIO pins 15, 17, and 31 each to one scope channel.

In [25]:
# SeLect HD channel grouping.
hd.set_channel_grouping(0)

# Enable output 0 used for triggering.
hd.enable_output(0)

In [26]:
# This sequence will send out a trigger and execute the commands  dig_pulse (yet to be specified).
sequence_txt = """\

        while (1) {

          // Trigger to scope.
          setTrigger(1);
          
          _dig_pulse_


          // Wait
          wait(1000);


          setTrigger(0);

        }
        """

# Create Sequence instance.
seq = Sequence(hd, sequence_txt, ['dig_pulse'])

# Replace dig_pulse placeholder by digital pulse sequence instruction set.
seq.replace_placeholder('dig_pulse', dig_pulse_sequence)

In [27]:
# Create an instance of the AWG Module.
awg = AWGModule(hd, 0)
awg.set_sampling_rate('2.4 GHz') # Set 2.4 GHz sampling rate.

# Upload sequence.
if awg is not None:
    awg.compile_upload_sequence(seq)

In [28]:
# Now we're almost ready, we only have to make sure that the 8 bit buses for bits 15, 17, and 31 are driven. 
# This can be done automatically by calling the following:

pb_handler.setup_hd()

In [31]:
# Start the AWG
awg.start()

The output waveform can be verified on the scope, we have successfully reproduced our digital rabi pulse sequence:

![alt text](pictures/pulse_sequence.jpg "Title")


In [30]:
# Let's compare to the target pulse sequence.
iplot(rabi_pulse)